In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import math
from collections import Counter
import operator

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/data-work/data_work


Рекомендация по словам. Есть несколько основных шагов. Первое нужно сделать чистый review, после сделаю два датасета, маленький и большой чтоб сравнивать результаты и в конце приступлю к рекомендациям.

In [2]:
data = pd.read_csv("/kaggle/input/data-work/data_work")
data
data['product_Count'] = data['asin'].map(data['asin'].value_counts())
data['Reviewer_Count'] = data['reviewerID'].map(data['reviewerID'].value_counts())

In [3]:
data['reviewText']=data['reviewText'].str.lower()
import string
string.punctuation 
def remove_punctuation(text):
    '''this function splits the words to letters, remove punctuation and 
        again getting together letters as words '''
    text_no_punc = "".join([c for c in text if c not in string.punctuation])
    return text_no_punc
data['review_clean'] = data['reviewText'].apply(lambda x: remove_punctuation(x))
data.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,product_Count,Reviewer_Count,review_clean
0,5.0,True,"11 9, 2012",A2M1CU2IRZG0K9,0005089549,{'Format:': ' VHS Tape'},Terri,so sorry i didn't purchase this years ago when...,Amazing!,1352419200,2,1,so sorry i didnt purchase this years ago when ...
1,5.0,True,"12 30, 2011",AFTUJYISOFHY6,0005089549,{'Format:': ' VHS Tape'},Melissa D. Abercrombie,believe me when i tell you that you will recei...,Great Gospel VHS of the Cathedrals!,1325203200,2,1,believe me when i tell you that you will recei...
2,5.0,True,"04 21, 2005",A3JVF9Y53BEOGC,000503860X,{'Format:': ' DVD'},Anthony Thompson,"i have seen x live many times, both in the ear...",A great document of a great band,1114041600,5,2,i have seen x live many times both in the earl...
3,5.0,True,"04 6, 2005",A12VPEOEZS1KTC,000503860X,{'Format:': ' DVD'},JadeRain,"i was so excited for this! finally, a live co...",YES!! X LIVE!!,1112745600,5,9,i was so excited for this finally a live conc...
4,5.0,True,"12 3, 2010",ATLZNVLYKP9AZ,000503860X,{'Format:': ' DVD'},T. Fisher,x is one of the best punk bands ever. i don't ...,X have still got it,1291334400,5,1,x is one of the best punk bands ever i dont ev...


In [4]:
# сделаем 2 датасета большой и не очень
# у одного будет 10000 строк у другого 100000
data_small = data.sample(10000)
data_big = data.sample(100000)
stop = set(stopwords.words("english"))
data_small['word_without_stop'] = data_small['review_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data_big['word_without_stop'] = data_big['review_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [5]:
data_small = data_small.reset_index(drop=True)


In [6]:
data_big = data_big.reset_index(drop=True)

In [7]:
data_big

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,product_Count,Reviewer_Count,review_clean,word_without_stop
0,5.0,True,"05 31, 2014",A1LTDQ6UQGX8WC,B000CDWJTK,{'Size:': ' 12 In x 50 Ft'},JB,just the right size to line a box or wrap your...,Excellent,1401494400,1,2,just the right size to line a box or wrap your...,right size line box wrap items easy cut shape ...
1,2.0,True,"07 27, 2013",A3PHK2TCCQUZA9,6303257852,{'Format:': ' Amazon Video'},DK,my 5 year old loves old godzilla movies or i w...,You have to be 5,1374883200,107,1,my 5 year old loves old godzilla movies or i w...,5 year old loves old godzilla movies would nev...
2,5.0,True,"06 15, 2014",A16GPIFIF0O0QT,0783229569,{'Format:': ' Amazon Video'},DanielJ,"this movie is hilarious, depp and del toro exc...",2 hours of non stop drug abuse,1402790400,509,1,this movie is hilarious depp and del toro exci...,movie hilarious depp del toro excite roles alm...
3,4.0,True,"08 24, 2006",A3UPYGJKZ0XTU4,6301863186,{'Format:': ' DVD'},mirasreviews,"""house of strangers"" is based on the novel ""i'...",The Immigrant Experience as Greek Tragedy.,1156377600,44,54,house of strangers is based on the novel ill n...,house strangers based novel ill never go pulit...
4,5.0,True,"06 5, 2009",A2E5LZA5N5YNRC,078322947X,{'Format:': ' DVD'},Iraq Combat Vet,"ah, a great film as a combat veteran of iraq ,...",Great film,1244160000,236,2,ah a great film as a combat veteran of iraq i...,ah great film combat veteran iraq understand i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4.0,True,"06 28, 2012",A28T2GJMPK645X,6303072798,{'Format:': ' DVD'},Paula,i first saw this film as a child and i was hap...,a winner--and one you remember,1340841600,170,1,i first saw this film as a child and i was hap...,first saw film child happy find vhs years ago ...
99996,5.0,True,"07 28, 2014",AQBRR027FZA65,B000AXZ9DK,"{'Size Name:': ' .155"" Spline Size', 'Style:':...",Araf Nawr,nice quality build on the screen and installat...,Nice quality build on the screen and installat...,1406505600,12,2,nice quality build on the screen and installat...,nice quality build screen installation fine in...
99997,5.0,True,"10 8, 2014",A8YH54JTIC8QU,B00137HJPS,{'Package Quantity:': ' 1'},GRM,"light, sharp and easy to get to. i enjoy the d...",My favorite EDC so far!,1412726400,9,1,light sharp and easy to get to i enjoy the dee...,light sharp easy get enjoy deep pocket clip cl...
99998,5.0,True,"01 2, 2015",ASSUR7Q0YT24C,078322740X,{'Format:': ' Amazon Video'},dogface boy,this is a great story if you like steve martin...,This is a great story if you like Steve Martin...,1420156800,170,2,this is a great story if you like steve martin...,great story like steve martin goldie hawn town...


In [8]:
def cosine_similarity_of(text1, text2):
        
        # taking words
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        # creating dictionary from each word and count
        vector1 = Counter(first)
        vector2 = Counter(second)

        # changing vectors to sets to find comman words with intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            # getting amount of each common word for vectors and multiply them also add them
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        # getting summation of word counts for every vector
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        # calculating magnitude of vector with squared sums
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
            return 0.0
        else:
            return float(dot_product) / magnitude

In [9]:
data_small['word_without_stop']

0                               great price fast delivery
1                                                   works
2                                        great kids movie
3       buy movie coz loved back come tv fun watch two...
4                      much cheaper hp cartridges go long
                              ...                        
9995    really lovely wedding card raised flowers imit...
9996    fun watch husband laughed lot goldie hawn stev...
9997    found instant video chance weeks ago friend de...
9998                                          great movie
9999    good drama interesting suspense plot slightly ...
Name: word_without_stop, Length: 10000, dtype: object

In [10]:
text1 = data_small['word_without_stop'][26]
text2 = data_small['word_without_stop'][27]
cosine_similarity_of(text1, text2)

0.2182178902359924

In [11]:
def get_recommendations(df,keywords):

        score_dict = {}

        for index, row in df.iterrows():
            score_dict[index] = cosine_similarity_of(row['word_without_stop'], keywords)

        # sorting books according to text similarities
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin','word_without_stop', 'score'))

        # taking highest 10 scored books
        for i in sorted_scores:
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 
                                        'asin': df.iloc[i[0]]['asin'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'], 
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9: #setting top-10/ can change according to wanted recommendation number
                break

        return result

In [12]:
keywords = "best movie"
top_10= get_recommendations(data_big,keywords)
top_10

/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarnin

,reviewerID,asin,word_without_stop,score
0,A9BJUYI09FUK9,5554452574,best movie,1.000000
1,A38MQ3RH7071SC,0788821075,best movie,1.000000
2,AVPH7U905TW2E,6302999790,best movie,1.000000
3,A35MSQSDFQEN4S,630150528X,best best awesome movie,0.866025
4,A37LX2TUM7HAMJ,6302483395,best movie ever,0.816497
5,A3O87NSZ3Y8PWL,6303212379,best movie ever,0.816497
6,A3ETC4XXBPPZDT,0783227361,best movie ever,0.816497
7,A2RAOKQM7JYP75,1562550888,best movie ever,0.816497
8,A2GT3LT6TLVRD2,6302637899,best movie ever,0.816497
9,A1RG3F2C79DJNZ,0788821075,best movie ever,0.816497


In [13]:
keywords = "great history dance movies"
top_10= get_recommendations(data_big,keywords)
top_10

/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarnin

,reviewerID,asin,word_without_stop,score
0,A1BIKI7NQAUM2F,0788812467,great movies,0.707107
1,A36Z91ST5J2RRU,6300183513,great movies,0.707107
2,A1FEN7BDQIPW3M,6302088593,great movies,0.707107
3,A3QO77XR7RJV23,0793906091,great movies,0.707107
4,ARJ7C3EOG3PIV,6303131891,movies great,0.707107
5,A2490GBIRZVVJT,6302098440,great movies,0.707107
6,A2BBDOWF6JX0VP,6303165680,great movies,0.707107
7,ARF601E834COA,0793906091,great movies,0.707107
8,A2WC7IZ886XP1J,0793906091,great movies,0.707107
9,A2Y17GIO1XDLGC,0792845765,great movies,0.707107


In [14]:
keywords = "titanic"
top_10= get_recommendations(data_big,keywords)
top_10

/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'],
/tmp/ipykernel_20/2168849447.py:18: FutureWarnin

,reviewerID,asin,word_without_stop,score
0,A3OFJES10PZGEH,0792151712,movie romance leo kate adventure titanic sinki...,0.668153
1,A2JW7VFF9H01V5,0792151712,love movie skn finally something titanic deser...,0.552345
2,A36KRQ9O0IK36M,0792151712,used school lesson titanic,0.500000
3,A31GG0IGRKLVTA,0792151712,titanic one greatest movies,0.500000
4,A3S7WU6DIH71CC,0792151712,cant get enough whole titanic thing 4 dvds soo...,0.470592
5,A3I2NQYN7R9Z8W,156501345X,back mid1990s documentary came box set vhs for...,0.467888
6,A27IURIXX9JW7C,0792151712,yes recivce titanic last week,0.447214
7,AL32HGC4TBIIB,0792151712,wonderful watching titanic 3dif havent seen ti...,0.436436
8,A3LC2I812OWCKB,0792151712,although doomed rms titanic sunk compelling ro...,0.419591
9,A14U40MX3CPO8Q,0792151712,parts film may little overdramatic titanic emo...,0.393919


Теперь хочется изменить немного эффект рейтинга и учитывать также рейтинг.

In [15]:
def rating_weight(rating, b):
    '''calculating rating weight'''
    w = (2*b/5)*rating - b
    return w

In [16]:
def calculate_final_score(cos_sim, w):
        '''adding weight effect to cosine similarity'''
        effect = (cos_sim / 100) * w
        return cos_sim + effect

In [17]:
def get_recommendations_rat(df,keywords):
        '''getting recommendations with rating effect'''
        score_dict = {}

        for index, row in df.iterrows():
            cs_score = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['overall']
            
            # I choose Q as 10 to not to change weight too much
            rating_effect = rating_weight(rating,10)

            last_score = calculate_final_score(cs_score, rating_effect)

            score_dict[index] = last_score
            
        # sorting books according to text similarities
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin','word_without_stop','overall', 'score'))

         # taking highest 10 scored books
        for i in sorted_scores:
        
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'],
                                        'overall': df.iloc[i[0]]['overall'],
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9:
                break
                
        return result

In [18]:
keywords = "titanic"
top_10= get_recommendations_rat(data_small,keywords)
top_10

/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append

,reviewerID,asin,word_without_stop,overall,score
0,A37J6JRKISQMH0,0792151712,titanic excellent example grand moviemaking fi...,5.0,0.307941
1,ACO20QL72D1E6,0792151712,account ship demise 2 got romeo julietstory on...,5.0,0.281681
2,A3R60L8WL8VG88,0792151712,titanic best romantic drama thriller movie mov...,5.0,0.259272
3,A2YA928MHO2V6X,0792151712,titanic years parody stale titanic still rembe...,3.0,0.253031
4,AS5OGRXQ4T7BY,0792151712,everything could say titanic said many times e...,5.0,0.241481
5,A3PAHM8VNML9F6,1558908676,imax dvd disappointing much consists old newsr...,1.0,0.236858
6,A1G9FX1KV45N41,0792151712,one greatest romantic adventures 1990s involvi...,5.0,0.229366
7,AKS9SJTHTU0OM,0792151712,ton problems titanic cartoon villains stilted ...,4.0,0.213068
8,A3DDPQ9342UNMY,0792151712,theres nothing wrong telling love story backgr...,5.0,0.212132
9,A16X61RLY1YJL7,0792151712,incredible films winning eleven academy awards...,5.0,0.178885


In [19]:
top_10= get_recommendations_rat(data_big,keywords)
top_10

/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3019117901.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append

,reviewerID,asin,word_without_stop,overall,score
0,A3OFJES10PZGEH,0792151712,movie romance leo kate adventure titanic sinki...,4.0,0.708242
1,A2JW7VFF9H01V5,0792151712,love movie skn finally something titanic deser...,5.0,0.607579
2,A31GG0IGRKLVTA,0792151712,titanic one greatest movies,5.0,0.550000
3,A36KRQ9O0IK36M,0792151712,used school lesson titanic,4.0,0.530000
4,A3S7WU6DIH71CC,0792151712,cant get enough whole titanic thing 4 dvds soo...,5.0,0.517651
5,A3I2NQYN7R9Z8W,156501345X,back mid1990s documentary came box set vhs for...,5.0,0.514676
6,A27IURIXX9JW7C,0792151712,yes recivce titanic last week,5.0,0.491935
7,AL32HGC4TBIIB,0792151712,wonderful watching titanic 3dif havent seen ti...,5.0,0.480079
8,A3LC2I812OWCKB,0792151712,although doomed rms titanic sunk compelling ro...,5.0,0.461550
9,A14U40MX3CPO8Q,0792151712,parts film may little overdramatic titanic emo...,5.0,0.433311


Если смотреть то немного таблица все же поменялась и значения в ней тоже

## Учитываем количество отзывов на товар

In [20]:
math.exp(-0.68) 

0.5066169923655895

In [21]:
def get_rating_weight_with_threshold(rating, count, threshold, b):
            # rating effect 
            w = (2*b/5)*rating - b
            # multiplier effect
            M = math.exp((-threshold*0.68)/count)

            return w * M

In [22]:
def get_recommendations_rat_count(df,keywords):

        score_dict = {}

        for index, row in df.iterrows():
            cs = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['overall']
            rating_count = row['product_Count']
            threshold = 110
            # I choose Q as 10 to not to change weight too much
            rating_contribution = get_rating_weight_with_threshold(rating,rating_count,threshold,10)

            last_score = calculate_final_score(cs, rating_contribution)

            score_dict[index] = last_score
            
        # sorting books according to text similarities
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin','word_without_stop','overall', 'score'))

        # taking highest 10 scored books
        for i in sorted_scores:
            
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'],
                                        'overall': df.iloc[i[0]]['overall'],
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9:
                break
                
        return result

In [23]:
keywords = "titanic"
top_10= get_recommendations_rat_count(data_big,keywords)
top_10

/tmp/ipykernel_20/2693832237.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/2693832237.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/2693832237.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/2693832237.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append

,reviewerID,asin,word_without_stop,overall,score
0,A3OFJES10PZGEH,0792151712,movie romance leo kate adventure titanic sinki...,4.0,0.706307
1,A2JW7VFF9H01V5,0792151712,love movie skn finally something titanic deser...,5.0,0.604913
2,A31GG0IGRKLVTA,0792151712,titanic one greatest movies,5.0,0.547587
3,A36KRQ9O0IK36M,0792151712,used school lesson titanic,4.0,0.528552
4,A3S7WU6DIH71CC,0792151712,cant get enough whole titanic thing 4 dvds soo...,5.0,0.515380
5,A27IURIXX9JW7C,0792151712,yes recivce titanic last week,5.0,0.489776
6,AL32HGC4TBIIB,0792151712,wonderful watching titanic 3dif havent seen ti...,5.0,0.477973
7,A3I2NQYN7R9Z8W,156501345X,back mid1990s documentary came box set vhs for...,5.0,0.468462
8,A3LC2I812OWCKB,0792151712,although doomed rms titanic sunk compelling ro...,5.0,0.459525
9,A14U40MX3CPO8Q,0792151712,parts film may little overdramatic titanic emo...,5.0,0.431410


Как видно поменялись немного значения

## Придание веса отзывам которые оценены высокими значениями
Для позитивных оценок будет новый рейтинг то есть к рейтингу прибавят (rating+7)/2, а негативные просто поделят на 2

In [24]:
data_small['positive'] = data_small['overall'].apply(lambda x: True if x>=3 else False)
data_big['positive'] = data_big['overall'].apply(lambda x: True if x>=3 else False)

In [25]:
def get_recommendations_rat_count_pos(df,keywords):

        score_dict = {}

        for index, row in df.iterrows():
            cs = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['overall']
            rating_count = row['product_Count']
            
            if row['positive']==True:
                pos_rat = (7+rating)/2
            else:
                pos_rat = rating/2
                
            threshold=110
            # I choose Q as 10 to not to change weight too much
            rating_contribution =get_rating_weight_with_threshold(pos_rat,rating_count,threshold,10)

            last_score = calculate_final_score(cs, rating_contribution)
            # keeping scores
            score_dict[index] = last_score
            
        #sort cities by score and index.
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        # building new df to keep values
        result = pd.DataFrame(columns=('reviewerID', 'asin','word_without_stop','overall', 'score'))

        # taking highest 10 scored books
        for i in sorted_scores:
            
            result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
                                        'word_without_stop': df.iloc[i[0]]['word_without_stop'],
                                        'overall': df.iloc[i[0]]['overall'],
                                        'score': i[1]}, ignore_index=True)
            counter += 1

            if counter>9:
                break
                
        return result

In [26]:
keywords = "titanic"
top_10= get_recommendations_rat_count_pos(data_big,keywords)
top_10

/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append

,reviewerID,asin,word_without_stop,overall,score
0,A3OFJES10PZGEH,0792151712,movie romance leo kate adventure titanic sinki...,4.0,0.744461
1,A2JW7VFF9H01V5,0792151712,love movie skn finally something titanic deser...,5.0,0.625941
2,A31GG0IGRKLVTA,0792151712,titanic one greatest movies,5.0,0.566621
3,A36KRQ9O0IK36M,0792151712,used school lesson titanic,4.0,0.557104
4,A3S7WU6DIH71CC,0792151712,cant get enough whole titanic thing 4 dvds soo...,5.0,0.533295
5,A27IURIXX9JW7C,0792151712,yes recivce titanic last week,5.0,0.506801
6,AL32HGC4TBIIB,0792151712,wonderful watching titanic 3dif havent seen ti...,5.0,0.494588
7,A3LC2I812OWCKB,0792151712,although doomed rms titanic sunk compelling ro...,5.0,0.475498
8,A3I2NQYN7R9Z8W,156501345X,back mid1990s documentary came box set vhs for...,5.0,0.468692
9,A14U40MX3CPO8Q,0792151712,parts film may little overdramatic titanic emo...,5.0,0.446406


In [27]:
top_10= get_recommendations_rat_count_pos(data_small,keywords)
top_10

/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'reviewerID': df.iloc[i[0]]['reviewerID'], 'asin': df.iloc[i[0]]['asin'],
/tmp/ipykernel_20/3422775811.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append

,reviewerID,asin,word_without_stop,overall,score
0,A37J6JRKISQMH0,0792151712,titanic excellent example grand moviemaking fi...,5.0,0.317247
1,ACO20QL72D1E6,0792151712,account ship demise 2 got romeo julietstory on...,5.0,0.290194
2,A2YA928MHO2V6X,0792151712,titanic years parody stale titanic still rembe...,3.0,0.271679
3,A3R60L8WL8VG88,0792151712,titanic best romantic drama thriller movie mov...,5.0,0.267108
4,A3PAHM8VNML9F6,1558908676,imax dvd disappointing much consists old newsr...,1.0,0.251976
5,AS5OGRXQ4T7BY,0792151712,everything could say titanic said many times e...,5.0,0.248779
6,A1G9FX1KV45N41,0792151712,one greatest romantic adventures 1990s involvi...,5.0,0.236297
7,AKS9SJTHTU0OM,0792151712,ton problems titanic cartoon villains stilted ...,4.0,0.223964
8,A3DDPQ9342UNMY,0792151712,theres nothing wrong telling love story backgr...,5.0,0.218543
9,A16X61RLY1YJL7,0792151712,incredible films winning eleven academy awards...,5.0,0.184291


Значения, что в большом что в маленьком массиве поменялись и это можно заметить сравнив до и после. 